Hypothetical interaction

In [64]:
import sys
import os

# Add actrme to path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath("."), '..', 'actrme')))
from importlib import reload
import actrme.models.model
import actrme.basic
from actrme.modules.declarative import DeclarativeMemory
import pandas as pd

reload(actrme.basic)
reload(actrme.models.model)


<module 'actrme.model' from '/Users/andreastocco/Documents/Research/ACTR-ME/actrme/model.py'>

# Examples

Here is a list of examples.

## 1. A Memory Experiment

We will start with a memory experoiment. The experiment is taken from a paper by Xu et al. (2025), and the original data is available on OSF.

The experiment used a retrieval practice procedure: Participants are presented with simple paired associates, e.g., a pair of Swahili-English words, such as "samaki/fish". Each pair is presented during a study trial, and then probed in a series of test trials.

In [65]:
data = pd.read_csv("../data/memory_experiment.csv")
data

,subj,item,resp,RT,type,time,acc
0,1501,3,fish,2.295,study,3.131,1
1,1501,3,fish,0.888,test,5.426,1
2,1501,12,book,3.128,study,10.248,1
3,1501,12,book,0.841,test,12.776,1
4,1501,2,beer,4.203,study,18.824,1
...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1
147,1501,5,office,1.025,test,715.292,1
148,1501,12,book,0.975,test,717.400,1
149,1501,16,work,1.042,test,719.503,1


We first need to add two columns where the model will store its predictions

In [66]:
data['predicted_resp'] = ""
data['predicted_rt'] = 0.0

The dataset is not quite ready for analysis yet. To run this analysis, we need to add a column that contains the correct response for each item. To do so, we first need to create a dictoriary of all the responses for each 'study' trial; these are the correct pair-associates.

In [67]:
study = data[data['type'] == 'study']
mappings = dict(zip(study.item, study.resp))
mappings

{3: 'fish',
 12: 'book',
 2: 'beer',
 9: 'police',
 22: 'fruit',
 5: 'office',
 14: 'voice',
 23: 'pain',
 8: 'music',
 10: 'cloud',
 18: 'friend',
 11: 'chicken',
 25: 'language',
 15: 'thanks',
 20: 'bike',
 7: 'eye',
 16: 'work',
 19: 'wheel',
 13: 'song',
 4: 'pants',
 21: 'color',
 6: 'monkey',
 24: 'shop',
 1: 'enemy',
 17: 'holidays'}

Now we can add a new column, 'correct', that contains the correct response for each item presented, whether the participant responded correctly or not.

In [68]:
data['correct'] = data['resp']
for index, row in data.iterrows():
    if row['type'] == "test":
        data.at[index, 'correct'] = mappings[row['item']]
    #elif row['type'] == "study":
    #    data.at[index, 'RT'] = np.nan

data

,subj,item,resp,RT,type,time,acc,predicted_resp,predicted_rt,correct
0,1501,3,fish,2.295,study,3.131,1,,0.0,fish
1,1501,3,fish,0.888,test,5.426,1,,0.0,fish
2,1501,12,book,3.128,study,10.248,1,,0.0,book
3,1501,12,book,0.841,test,12.776,1,,0.0,book
4,1501,2,beer,4.203,study,18.824,1,,0.0,beer
...,...,...,...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1,,0.0,cloud
147,1501,5,office,1.025,test,715.292,1,,0.0,office
148,1501,12,book,0.975,test,717.400,1,,0.0,book
149,1501,16,work,1.042,test,719.503,1,,0.0,work


This is an alteration of "study" and "test" trials. 

### Creating a model

To fit the data, we need to create a simple model. Fitting the data means that the model is run trial by trial to maximize the probability of responding exactly like the participant.

To do so, we need to create a `DataModel`, which is a generic name for a model simulates behavior or performance "tracing".

In [69]:
reload(actrme.models.model)
model = actrme.models.model.DataModel(data)
model

<(..Num) 'time'=0>


Now, we have not yet specified how the model works. In this example, we will create a simple model that does only one thing: encodes things on study trials, and attempts to retrieve them on test trials. To do so, we need to add a new cognitive function to the model, Declarative Memory. 

In [70]:
reload(actrme.modules.declarative)
dm = DeclarativeMemory()
print(dm)
model.add_module(dm)

<(..Sym) 'encode'={}>
<(..Sym) 'cue'={}>
<GenericModule [module]>


In [71]:
print(model.outputs)

[<(Num..) 'rt'=0>, <(Sym..) 'retrieval'={}>, <(Num..) 'retrieval time'=0>, <(Num..) 'retrieval probability'=0>]


In [72]:
print(model.inputs)

[<(..Num) 'time'=0>, <(..Sym) 'encode'={}>, <(..Sym) 'cue'={}>]


In [73]:
model.connect_input("item", model.get_input("encode"))
model.connect_input("time", model.get_input("time"))
model.connect_input("resp", model.get_input("encode"))

In [74]:
# We want to simply maximize the probability of a response

model.connect_output("predicted_resp", model.get_output("retrieval"))

In [75]:
# Now, run the model
model.run()



# Now fit:
model.fit()


RecursionError: maximum recursion depth exceeded while calling a Python object

In [76]:
data

,subj,item,resp,RT,type,time,acc,predicted_resp,predicted_rt,correct
0,1501,3,fish,2.295,study,3.131,1,,0.0,fish
1,1501,3,fish,0.888,test,5.426,1,,0.0,fish
2,1501,12,book,3.128,study,10.248,1,,0.0,book
3,1501,12,book,0.841,test,12.776,1,,0.0,book
4,1501,2,beer,4.203,study,18.824,1,,0.0,beer
...,...,...,...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1,,0.0,cloud
147,1501,5,office,1.025,test,715.292,1,,0.0,office
148,1501,12,book,0.975,test,717.400,1,,0.0,book
149,1501,16,work,1.042,test,719.503,1,,0.0,work
